## Module 5: General Linear Regression and Statistical Inference

### Step 0

Load the appropriate libraries and bring in the data. Note that we have to run a script to get the [California Housing dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html) to match as it is in scikit-learn. We cannot pull it directly from scikit-learn since CodeGrade cannot access the internet.

In [1]:
# CodeGrade step0

from sklearn.datasets import fetch_california_housing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
import os
import tarfile
import joblib # Import joblib directly
from sklearn.datasets._base import _pkl_filepath, get_data_home
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns

archive_path = "California Housing.tgz" # change the path if it's not in the current directory
data_home = get_data_home(data_home=None) # change data_home if you are not using ~/scikit_learn_data
if not os.path.exists(data_home):
    os.makedirs(data_home)
filepath = _pkl_filepath(data_home, 'cal_housing.pkz')

with tarfile.open(mode="r:gz", name=archive_path) as f:
    cal_housing = np.loadtxt(
        f.extractfile('CaliforniaHousing/cal_housing.data'),
        delimiter=',')
    # Columns are not in the same order compared to the previous
    # URL resource on lib.stat.cmu.edu
    columns_index = [8, 7, 2, 3, 4, 5, 6, 1, 0]
    cal_housing = cal_housing[:, columns_index]

    joblib.dump(cal_housing, filepath, compress=6) # Now using the directly imported joblib


# Load the dataset
california = fetch_california_housing(as_frame=True)
data = california.data
data['MedianHouseValue'] = california.target

Print the basic information of the data using `.info()` and `.describe`.

In [2]:
# Display basic information
df = data.copy()
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MedInc            20640 non-null  float64
 1   HouseAge          20640 non-null  float64
 2   AveRooms          20640 non-null  float64
 3   AveBedrms         20640 non-null  float64
 4   Population        20640 non-null  float64
 5   AveOccup          20640 non-null  float64
 6   Latitude          20640 non-null  float64
 7   Longitude         20640 non-null  float64
 8   MedianHouseValue  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB
None
             MedInc      HouseAge      AveRooms     AveBedrms    Population  \
count  20640.000000  20640.000000  20640.000000  20640.000000  20640.000000   
mean       3.870671     28.639486      5.429000      1.096675   1425.476744   
std        1.899822     12.585558      2.474173      0.473911   1132.462122   
min        0.4

### Step 1

Let `X` be the variables `MedInc`, `AveRooms`, and `AveOccup` and add the constant for the intercept. Let `y` be the `MedianHouseValue`.

Now fit the regreson model calling it `mlr_model`.

Finally, return the $r^2$ value of the model rounding to four decimal places.

In [5]:
# CodeGrade step1
X = df[['MedInc', 'AveRooms', 'AveOccup']]
y = df['MedianHouseValue']
X_const = sm.add_constant(X)
mlr_model = sm.OLS(y, X_const).fit()
round(mlr_model.rsquared, 4)


0.4808

Print the model summary.

In [6]:
# Print the model summary

print(mlr_model.summary())

                            OLS Regression Results                            
Dep. Variable:       MedianHouseValue   R-squared:                       0.481
Model:                            OLS   Adj. R-squared:                  0.481
Method:                 Least Squares   F-statistic:                     6370.
Date:                Thu, 22 May 2025   Prob (F-statistic):               0.00
Time:                        12:27:43   Log-Likelihood:                -25477.
No. Observations:               20640   AIC:                         5.096e+04
Df Residuals:                   20636   BIC:                         5.099e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6069      0.016     37.444      0.0

### Step 2

Let `p_values` be the models' p-values.

Return the four p-values using `.iloc[]` from the first value to the fourth, in order and separated by commas. Make sure to round each to 5 decimal places.

In [10]:
# CodeGrade step2
p_values = mlr_model.pvalues
rounded_p_values = [round(p, 5) for p in p_values.iloc[0:4]]
print(rounded_p_values, sep=', ')


[0.0, 0.0, 0.0, 0.0]


### Step 3

Identify the significant predictors (strictly less than $\alpha=0.05$) calling this `significant_predictors`.

Reutn the shape of `significant_predictors`.

In [11]:
# CodeGrade step3
significant_predictors = p_values[p_values < 0.05]
significant_predictors.shape


(4,)

### Step 4

Find the confidence intervals of the model (at a 95% level of confidence) and calling this `conf_intervals`.

Using `.iloc[,]` and rounding to 2 decimal places return the four confidence intervals in order of (separated by commas)

> first row and first column, first row and second column, second row and first column, second row and second column





In [12]:
# CodeGrade step4
conf_intervals = mlr_model.conf_int()
value_1 = round(conf_intervals.iloc[0, 0], 2)  # first row, first column
value_2 = round(conf_intervals.iloc[0, 1], 2)  # first row, second column
value_3 = round(conf_intervals.iloc[1, 0], 2)  # second row, first column
value_4 = round(conf_intervals.iloc[1, 1], 2)  # second row, second column

print(value_1, value_2, value_3, value_4, sep=', ')


0.58, 0.64, 0.43, 0.44


Now to see how the intervals looks "nicely" return `conf_intervals`.

In [13]:
#Pretty CIs
conf_intervals


,0,1
const,0.575162,0.638703
MedInc,0.428363,0.441003
AveRooms,-0.043178,-0.033474
AveOccup,-0.005266,-0.003081


### Step 5

Add a quadratic term to the model, calling the new model `quad_model` where a new term is added to the data, viz. `MedInc_squared`, which is the square of `MedInc`.

Return $r^2$ of the quadratic model rounded to four decumal places.

In [14]:
# CodeGrade step5
df['MedInc_squared'] = df['MedInc'] ** 2
X_quad = df[['MedInc', 'MedInc_squared', 'AveRooms', 'AveOccup']]
y_quad = df['MedianHouseValue']
X_quad_const = sm.add_constant(X_quad)
quad_model = sm.OLS(y_quad, X_quad_const).fit()
round(quad_model.rsquared, 4)


0.4858

Now print the model summary.

In [15]:
# Print the model summary
print(quad_model.summary())

                            OLS Regression Results                            
Dep. Variable:       MedianHouseValue   R-squared:                       0.486
Model:                            OLS   Adj. R-squared:                  0.486
Method:                 Least Squares   F-statistic:                     4874.
Date:                Thu, 22 May 2025   Prob (F-statistic):               0.00
Time:                        12:31:55   Log-Likelihood:                -25378.
No. Observations:               20640   AIC:                         5.077e+04
Df Residuals:                   20635   BIC:                         5.081e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.3551      0.024     14.

### Step 6

Find the adjusted $r^2$ for both of the models and call them `adjusted_r2_base` and `adjusted_r2_quad`, respectively.

Return these two adjusted $r^2$'s rounded to four decimal places, separated by a comma.

In [16]:
# CodeGrade step6
adjusted_r2_base = round(mlr_model.rsquared_adj, 4)
adjusted_r2_quad = round(quad_model.rsquared_adj, 4)
print(adjusted_r2_base, adjusted_r2_quad, sep=', ')


0.4807, 0.4857


Print both these adjusted $r^2$'s.

In [ ]:
print("Adjusted R² (Baseline Model):", adjusted_r2_base)
print("Adjusted R² (Quadratic Model):", adjusted_r2_quad)